# Stage 1: Generate Stock Universe

- Gather stocks of interest
- Gather stocks from specific criteria (SP500 top 50...)
- Gather stocks from specific portfolio account
- Assemble stock universe 
- Use stock sentiment to select stocks
- Gather price histories

In [1]:
from platform import python_version
import time
from datetime import datetime
import os
import pandas as pd
import numpy as np
import math
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

%matplotlib inline
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (20, 8)

# Set the import path for the tools directiory
import sys
# insert at position 1 in the path, as 0 is the path of this file.
sys.path.insert(1, '../tools')
import importlib
import ameritrade_functions as amc
importlib.reload(amc)
import utils
importlib.reload(utils)

print(f'Python version: {python_version()}')
print(f'Pandas version: {pd.__version__}')

Python version: 3.8.10
Pandas version: 0.25.3


## Configure Ameritrade Information

Ameritrade credentials are stored in environment variables to keep from having unencrypted passwords stored on disk.

The module automatically masks the account numbers to protect the actual accounts. An Ameritrade user can have many investment accounts. We will be working with only one for this demonstration.

## Authentication Tokens

To get data from Ameritrade you will need to obtains a short time use token (there is a re-use token, but I have not coded it yet.) You only need to do this if you
are going to use an existing Ameritrade account to define an initial set of stocks to analyze.

To obtain a token, you will need to have a Chrome driver located somewhere on your system. This will allow the module to use your credentials to obtain an authentication token.

For security reasons, I sugges using environment variables to store your credential information. If you store them in property files, or just code them into your notebook, you risk sharing the information with others if you use GitHub or some other SCCS. This also makes it easier to have them availabe from project to project in your development environment

<span style="color:blue">Note: *Account numbers are masked for security purposes.*</span>

In [2]:
username = os.getenv('maiotradeuser')
password = os.getenv('maiotradepw')
client_id = os.getenv('maiotradeclientid')

# For Chromedriver
from pathlib import Path
chrome_executabel_path = str(Path.home()) + r'\Anaconda Projects\chromedriver\chromedriver'

# Make sure we have a data directory
Path('./data').mkdir(parents=True, exist_ok=True) 

# Which account are we interested in
masked_account_number = '#---9216'
account_portfolios_file_name = 'data/portfolio_data.csv'
portfolio_file_name = 'data/portfolio_' + masked_account_number[-4:] + '.csv'
price_histories_file_name = 'data/price_histories.csv'

## Stock Universe

Here we setup the univers. This needs some work. The long term goal is to use a pipeline process to help select stock that are in the top 500 or something similare.

For now we will use stocks from the portfolio, but stocks of interest (high news items), a list of well known stocks (this also has been augmented with some stocks that made Ameritrade's top 10 movers for a couple of days. This Ameritrade funciton has not been coded yet, but should be add down the line to automate pulling these tickers.

In [3]:
snp_500_df = utils.get_snp500()
snp_500_symbols = snp_500_df.index.to_list()
quote_dfs = []
for i in range(0, 500, 100):
    quote_dfs.append(amc.AmeritradeRest(username, password, client_id).get_quotes(snp_500_symbols[i:i+100]))
    
snp_500_quotes_df = pd.concat(quote_dfs, axis=0)
snp_500_quotes_df.describe()

snp_500_tickers = set(snp_500_df.index.to_list())
print(len(snp_500_tickers))

505


In [4]:
dow_tikers = set(utils.get_dow().index.to_list())
print(len(dow_tikers))

30


## Get sentiment data from Finvis

In [5]:
stock_universe = set.union(snp_500_tickers, dow_tikers)
print(len(stock_universe))

505


In [6]:
parsed_and_scored_news = utils.get_finvis_stock_sentiment(snp_500_symbols).sort_values(by='date')
parsed_and_scored_news

News Tables: 100%|████████████████████████████████████████████████████| 502/502 [00:02<00:00, 214.80News Table Items/s]


,ticker,date,time,headline,neg,neu,pos,compound
29106,L,2019-10-28,06:00AM,Loews Corporation Reports Net Income Of $72 Mi...,0.000,1.000,0.000,0.0000
29104,L,2019-10-28,09:00AM,Loews Corp. to Host Earnings Call,0.000,1.000,0.000,0.0000
29105,L,2019-10-28,06:11AM,Loews reports slump in third-quarter profit as...,0.000,0.775,0.225,0.4404
29101,L,2019-10-28,06:09PM,US Indexes Close Higher Monday,0.000,1.000,0.000,0.0000
29102,L,2019-10-28,03:26PM,Global Markets Advance Monday,0.000,1.000,0.000,0.0000
...,...,...,...,...,...,...,...,...
36421,PEP,2021-10-04,08:05AM,3 Stocks to Avoid This Week,0.355,0.645,0.000,-0.2960
36420,PEP,2021-10-04,08:05AM,"FedEx, Nike, Pepsi, Constellation and JPMorgan...",0.000,1.000,0.000,0.0000
6002,BAC,2021-10-04,08:00AM,Don't bank on fourth stimulus these stocks of...,0.000,1.000,0.000,0.0000
35107,OMC,2021-10-04,09:15AM,Credera Names Donal Smith UK Chief Client Officer,0.000,1.000,0.000,0.0000


In [7]:
# Group by date and ticker columns from scored_news and calculate the mean
mean_scores = parsed_and_scored_news.groupby(['ticker','date']).mean().fillna(0)
# Unstack the column ticker
mean_scores = mean_scores.unstack()
# Get the cross-section of compound in the 'columns' axis
mean_scores = mean_scores.xs('compound', axis="columns").transpose().fillna(0)
# Get cusmum score of each stock
cum_scores = mean_scores[-40:].cumsum(axis=0)
current_scores = cum_scores.iloc[-1]
mean_score = current_scores.mean()
stdv_score = current_scores.std()
cutoff = mean_score - stdv_score

print(mean_score, stdv_score, cutoff)

2.1384918713899106 1.70854346228345 0.4299484091064605


In [8]:
stock_universe = current_scores.where(current_scores > cutoff).dropna().index.to_list()

# Price History data

One you have a set of investments you want to work with, you will need to pull some historical data for them.

We will obtain 5 years of price histories. In the end this will provide us with 2 years of factor data since some of the factors are based on 1 year returns.

In [9]:
number_of_years = 5
price_histories = amc.AmeritradeRest(username, password, client_id).get_price_histories(stock_universe, datetime.today().strftime('%Y-%m-%d'), num_periods=number_of_years)
utils.save_price_histories(price_histories, price_histories_file_name)

Tickers:   0%|          | 0/437 [00:00<?, ?Price Histories/s]

In [10]:
price_histories.head()

,open,high,low,close,volume,ticker,date
0,47.07,47.30,46.670,46.83,1763985,A,2016-10-04
435016,50.45,50.54,49.765,49.92,1315850,RSG,2016-10-04
64794,36.36,36.42,35.700,35.77,2729968,BEN,2016-10-04
433757,64.14,64.69,64.020,64.26,1597497,ROST,2016-10-04
432498,182.28,182.69,180.750,181.95,524918,ROP,2016-10-04


In [11]:
price_histories = utils.read_price_histories(price_histories_file_name)
close = utils.get_close_values(price_histories)
close.tail()

ticker,A,AAL,AAP,AAPL,ABBV,ABC,ABT,ACN,ADBE,ADI,...,WYNN,XEL,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION,ZTS
date,,,,,,,,,,,,,,,,,,,,,
2021-09-28 00:00:00+00:00,159.900,21.43,214.78,141.91,107.340,121.290,118.74,324.21,578.7700,171.29,...,84.3100,61.42,59.92,59.14,128.920,122.54,150.85,531.18,62.74,192.77
2021-09-29 00:00:00+00:00,160.000,21.01,216.77,142.83,108.840,122.500,119.46,323.53,577.7000,168.84,...,83.6300,62.67,59.88,58.43,127.780,123.04,149.36,525.67,63.74,194.99
2021-09-30 00:00:00+00:00,157.530,20.52,208.89,141.50,107.870,119.450,118.13,319.92,575.7200,167.48,...,84.7500,62.50,58.82,58.05,123.680,122.31,146.36,515.42,61.89,194.14
2021-10-01 00:00:00+00:00,155.990,21.65,208.00,142.65,109.090,119.810,117.21,324.86,577.4700,167.37,...,87.2100,62.49,60.93,59.07,121.300,123.64,149.47,510.98,63.65,196.19
2021-10-04 00:00:00+00:00,155.405,21.78,208.05,140.95,109.601,120.185,117.14,322.99,571.0875,166.94,...,86.9487,62.90,61.72,59.41,121.445,124.06,149.35,508.08,64.04,193.98
